#실행전 준비

데이터 출처: https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=580

In [ ]:
!pip install -q -U transformers bitsandbytes protobuf
!pip install -q -U peft gradio langchain_chroma langchain_huggingface langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 147.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 25.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.33.2 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.33.2 which is incompatible.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.33.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 k

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files

src = list(files.upload().values())[0]
open('summerizer.py','wb').write(src)

Saving summerizer2.py to summerizer2.py


2930

In [ ]:
from google.colab import files

src = list(files.upload().values())[0]
open('predictor.py','wb').write(src)

Saving predictor2.py to predictor2.py


3582

In [ ]:
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

#라이브러리 import

In [ ]:
import pickle
import gradio as gr
from summerizer import summarize_text
from predictor import predict_law

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#test 데이터 로드

In [ ]:
with open("./final_test.pkl","rb") as fr:
    final_test = pickle.load(fr)

In [ ]:
type(final_test)

datasets.arrow_dataset.Dataset

In [ ]:
final_test[20]

{'messages': [{'content': '원고 측의 주장과 피고 측의 주장을 살펴보았을때, 내릴 수 있는 판결을 예측하여 관련 법조항과 결론을 작성해줘.',
   'role': 'system'},
  {'content': '사건유형 민사 세부유형 민사 2심 원고 측의 주장 요약 소외 3은 이 사건 부동산 매매대금 또는 차용금 1억 5천만 원을 변제하지 않아 원고에게 동산을 인도할 의무가 있으며, 이는 법인격 남용의 법리에 따른 채무 부담입니다. 또한, 소외 3은 2015년 11월 경 운영하던 개인사업체인 000의 재산을 소외 2와 함께 포괄적으로 양도하여 채무를 공동으로 부담하고 있습니다. 따라서 피고는 원고에게 위 채무를 공동으로 부담하여 140,754,000원을 지급해야 합니다. 피고 측의 주장 피고는 컨설팅회사의 조언으로 두진칼라팩을 법인으로 전환하여 세금 감면 등의 혜택을 받을 수 있다고 주장합니다. 이러한 상황에서 피고는 컨설팅회사의 법인 전환을 통해 유리한 조건으로 변제될 수 있으며, 이는 법적 지위를 확보하여 더 나은 결과를 기대할 수 있는 계기가 될 수 있습니다. 기초사실 요약 소외 1의 남편인 소외 2는 소외 1으로부터 2012년 10월에 매매대금을 지급하며 매매 계약을 체결했지만, 소외 2가 계약금을 지급하기 어려워하자 소외 1은 소외 2에게 1억 5천만원을 차용하고, 소외 3이 사업체를 운영할 예정이니 매수인 명의를 소외 3으로 변경하여 달라고 요청하여, 소외 1은 이 사건 부동산을 13억 원에 매도하고, 소외 3은 이 사건 토지 중 도로 지분 및 토목공사(아스콘)를 3억 3천만 원에 매도하는 매매계약을 체결했습니다. 이 과정에서 소외 3은 2013년 5월 16일에 소유권보존등기를 마쳤고, 2013년 8월 13일에 소유권이전등기를 마쳤으며, 2015년 11월 19일에 포괄양도 계약을 체결하여 소유권이전등기를 마쳤습니다. 소외 3은 2015년 12월 31일에 소유권이전등기를 마쳤지만, 같은 날 2015년 11월 22일에 신탁재산 귀속을

#프롬프트 제작 함수

In [ ]:
#gradio에서 받은 입력을 바탕으로 프롬프트 제작하는 함수
def make_prompt(court,case,detail_case,question, plaintiff, defendant, fact, conclusion):
    messages = {'messages':
                [
                    {'content': question, 'role': 'system'},
                    {'content': "사건유형 " + case + " 세부유형 " +  detail_case +  " " +  court +
                       " " +  plaintiff + " " + defendant + " " + fact
                       ,'role': 'user'},
                    {'content': conclusion,'role': 'assistant'}
                ]
               }
    return messages

#gradio 설정 및 실행

In [ ]:
with gr.Blocks() as demo:
    with gr.Column():
        with gr.Row():
            Dropdown1 = gr.Dropdown(["1심", "2심"], label =  "1심, 2심 선택", value="1심", interactive=True)
            Dropdown2 = gr.Dropdown(["민사", "형사", "행정"], label = "사건유형", value="민사", interactive=True)
            Dropdown3 = gr.Dropdown(["민사", "신청", "가사", "특허", "행정", "형사"], label ="세부유형", value="민사", interactive=True)
        with gr.Row():
            text1 = gr.Textbox(label="질문", lines=2)
        with gr.Row():
            text2 = gr.Textbox(label="원고", lines=5, interactive=True)
        with gr.Row():
            text3 = gr.Textbox(label="피고", lines=5, interactive=True)
        with gr.Row():
            text4 = gr.Textbox(label="기초사실", lines=5, interactive=True)
        with gr.Row():
            text5 = gr.Textbox(label="법원 판단", lines=5, interactive=True)

        with gr.Row():
            num = gr.Number(label="인덱스", interactive=True)
            example_btn = gr.Button("예시")
    with gr.Column():
        true_answer = gr.Textbox(label = "정답", lines=5)
        answer = gr.Textbox(label = "예측 결과", lines=5)
        rag = gr.Textbox(label = "참고자료", lines=5)
        answer_btn = gr.Button("변환")


    def example_prompt(n):
        idx = int(n)  # Number는 float로 들어오니까 안전하게 int 변환

        return (
            str(final_test[idx]['messages'][1]['content'][16:18]),
            str(final_test[idx]['messages'][1]['content'][5:7]),
            str(final_test[idx]['messages'][1]['content'][13:15]),
            final_test[idx]['messages'][0]['content'],
            "원고 측의 주장 " + final_test[idx]['messages'][1]['content'].split('피고 측의 주장')[0].split('원고 측의 주장')[1],
            "피고 측의 주장 " + final_test[idx]['messages'][1]['content'].split('피고 측의 주장')[1].split('기초사실')[0],
            "기초사실 " + final_test[idx]['messages'][1]['content'].split('기초사실')[1],
            final_test[idx]['messages'][2]['content'],
        )

    def run_predict(drop1, drop2, drop3, q, plaintiff, defendant, facts, court_judgment):
        """
        버튼 클릭 시 실행되는 함수.
        - 원고/피고/기초사실을 요약한 뒤
        - predict_law 클래스로 추론 실행
        """
        # 빈 문자열 방지용 처리(없어도 상관없긴함)
        plaintiff = plaintiff or ""
        defendant = defendant or ""
        facts = facts or ""
        court_judgment = court_judgment or ""

        # summarize_text 수행 (Summerizer의 summarize_text 사용)
        plaintiff_sum = summarize_text(plaintiff) if plaintiff.strip() else ""
        defendant_sum = summarize_text(defendant) if defendant.strip() else ""
        facts_sum = summarize_text(facts, 400) if facts.strip() else ""

        # predict_law에서 기대하는 데이터 포맷에 맞추기
        data = [
            drop1,           # 0: 1심/2심
            drop2,           # 1: 사건유형
            drop3,           # 2: 세부유형
            q,               # 3: 질문/시스템 프롬프트
            plaintiff_sum,   # 4: 원고 주장 (요약)
            defendant_sum,   # 5: 피고 주장 (요약)
            facts_sum,       # 6: 기초사실 (요약)
            court_judgment,  # 7: 법원 판단 (정답/참고용)
        ]

        predictor = predict_law(data)
        return predictor.prompt['messages'][2]['content'], predictor.answer, predictor.rag  # Gradio의 output으로 전달

    example_btn.click(#버튼 클릭시 test 데이터 샘플 출력
        fn=example_prompt,
        inputs=num,
        outputs=[Dropdown1,Dropdown2,Dropdown3,text1, text2, text3, text4, text5],
    ),
    answer_btn.click(#버튼 클릭시 gradio ui로 입력받은 데이터를 run_predict 함수로 넘겨서 모델 예측 실행
        fn=run_predict,
        inputs=[Dropdown1,Dropdown2,Dropdown3,text1, text2, text3, text4, text5],
        outputs=[true_answer, answer,rag]
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1c46f3c7d71269268c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
